In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

### 1. 재개발 횟수(원 크기)와 중앙값 거래가(색상도) 비교 분석

In [5]:
import pandas as pd
import folium
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm

# 1. 부동산 실거래가 데이터 처리 및 동-구 매핑 사전 생성
df_price = pd.read_csv('서울시 부동산 실거래가 정보(2022~2025).csv', encoding='cp949', low_memory=False)

# 동 이름 -> 구 이름 매핑 사전 (예: {"소공동": "중구", "공덕동": "마포구" ...})
# 실거래가 데이터에 법정동과 자치구 활용
dong_to_gu = df_price[['법정동명', '자치구명']].drop_duplicates().set_index('법정동명')['자치구명'].to_dict()

# 가격 데이터 중앙값 계산
df_price['물건금액(만원)'] = pd.to_numeric(df_price['물건금액(만원)'], errors='coerce')
price_stats = df_price.groupby('자치구명')['물건금액(만원)'].median().reset_index()
price_stats.columns = ['자치구명', '가격중앙값']

# 2. 정비사업 데이터 처리
df_plan = pd.read_csv("서울시 도시계획 정비사업 현황.csv", encoding="cp949")
df_plan["일자"] = df_plan["프로젝트코드"].str[8:16]
df_plan["구명"] = ""

for i in range(len(df_plan)):
    location_val = df_plan.loc[i, "위치명"]
    if pd.notnull(location_val) and str(location_val).strip() != "":
        loc_str = str(location_val).replace("서울특별시", "").replace("서울시", "").strip()
        
        #주소에서 '동'이나 '로' 이름을 추출하여 구명으로 변환, 구가 없는 결측치 데이터 보완
        find_idx = loc_str.find(" ")
        if find_idx == -1: find_idx = loc_str.find("동")
        if find_idx == -1: find_idx = loc_str.find("로")
        
        # 띄어쓰기나 '동/로' 앞의 글자 추출
        extracted_name = loc_str[:find_idx+1].strip() if find_idx != -1 else loc_str
        
        # 추출된 이름이 구라면 그대로 쓰고, 
        # 동 이름이라면 사전에서 구 이름을 찾아 사용.
        if "구" in extracted_name and len(extracted_name) <= 4:
            target_gu = extracted_name
        else:
            target_gu = dong_to_gu.get(extracted_name, "기타")
            
        df_plan.at[i, "구명"] = target_gu
    else:
        df_plan.at[i, "구명"] = "NULL"

# 2022년 이후 & 재개발 필터링, 소분류가 재개발이 아닌 행은 사용 안함.
redev_df = df_plan[(df_plan["일자"] >= "20220101") & (df_plan["소분류"].str.contains("재개발", na=False))].copy()

# 정상 집계
redev_counts = redev_df["구명"].value_counts().reset_index()
redev_counts.columns = ["자치구명", "재개발건수"]

# 3. 데이터 병합
final_map_df = pd.merge(redev_counts, price_stats, on="자치구명")

# 4. 지도 시각화
seoul_geo = {
    '강남구': [37.495, 127.062], '강동구': [37.549, 127.146], '강북구': [37.642, 127.011],
    '강서구': [37.561, 126.825], '관악구': [37.465, 126.943], '광진구': [37.546, 127.085],
    '구로구': [37.494, 126.856], '금천구': [37.460, 126.900], '노원구': [37.652, 127.075],
    '도봉구': [37.669, 127.032], '동대문구': [37.581, 127.054], '동작구': [37.502, 126.951],
    '마포구': [37.559, 126.908], '서대문구': [37.577, 126.939], '서초구': [37.473, 127.031],
    '성동구': [37.551, 127.041], '성북구': [37.605, 127.017], '송파구': [37.505, 127.115],
    '양천구': [37.524, 126.855], '영등포구': [37.522, 126.910], '용산구': [37.530, 126.980],
    '은평구': [37.619, 126.927], '종로구': [37.594, 126.977], '중구': [37.560, 126.995], '중랑구': [37.597, 127.092]
}

m = folium.Map(location=[37.563, 126.986], zoom_start=11, tiles='cartodbpositron')
norm = colors.Normalize(vmin=final_map_df['가격중앙값'].min(), vmax=final_map_df['가격중앙값'].max())
colormap = cm.get_cmap('YlOrRd')

for _, row in final_map_df.iterrows():
    gu = row['자치구명']
    if gu in seoul_geo:
        color_hex = colors.to_hex(colormap(norm(row['가격중앙값'])))
        radius_val = np.sqrt(row['재개발건수']) * 6 + 5 
        
        folium.CircleMarker(
            location=seoul_geo[gu],
            radius=radius_val,
            color=color_hex,
            fill=True,
            fill_color=color_hex,
            fill_opacity=0.7,
            popup=f"<b>{gu}</b><br>재개발: {row['재개발건수']}건<br>중앙값: {row['가격중앙값']:.0f}만원",
        ).add_to(m)
        
        folium.Marker(
            location=seoul_geo[gu],
            icon=folium.DivIcon(html=f'<div style="font-size: 9pt; font-weight: bold; color: black; text-align: center; white-space: nowrap; width: 100px; margin-left: -50px;">{gu}</div>')
        ).add_to(m)

m.save('seoul_redevelopment_complete_map.html')
m

### 2. 재개발 횟수와 상위 25% 거래가 비교 분석

In [11]:
import pandas as pd
import folium
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm
import re

# 1. 데이터 로드
# 파일명이 실제 환경과 다를 수 있으니 확인 바랍니다.
price_temp = pd.read_csv("서울시 부동산 실거래가 정보(2022~2025).csv", encoding="cp949", low_memory=False)
df_plan = pd.read_csv("서울시 도시계획 정비사업 현황.csv", encoding="cp949")

# 2. 법정동-자치구 매핑 사전 생성 (동 이름만 있을 때를 대비)
# 중복된 동 이름(예: 신사동) 중 가장 빈도가 높은 구를 선택하여 매핑의 일관성 유지
mapping_df = price_temp[['법정동명', '자치구명']].value_counts().reset_index()
dong_to_gu = mapping_df.drop_duplicates('법정동명').set_index('법정동명')['자치구명'].to_dict()

# 서울시 자치구 리스트 (정확한 매칭용)
seoul_gu_list = [
    '강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', 
    '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', 
    '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'
]

# 3. 구명 추출 함수 (안정성 강화)
def get_gu_name(location_val):
    if pd.isnull(location_val) or str(location_val).strip() == "":
        return "NULL"
    
    loc_str = str(location_val).replace("서울특별시", "").replace("서울시", "").strip()
    
    # [Step 1] 문자열 내에 자치구 이름이 직접 포함되어 있는지 확인 (가장 확실한 방법)
    for gu in seoul_gu_list:
        if gu in loc_str:
            return gu
            
    # [Step 2] 주요 동/로 이름 예외 매핑
    special_cases = {
        "영등포동": "영등포구", "노량진동": "동작구", "신림동": "관악구", 
        "중곡동": "광진구", "전농동": "동대문구", "금호동": "성동구", 
        "길음동": "성북구", "상계동": "노원구", "수색": "은평구", "명륜": "종로구", "한강로": "용산구"
    }
    for key, val in special_cases.items():
        if key in loc_str:
            return val

    # [Step 3] 동 이름을 정규표현식으로 추출하여 사전 검색
    match = re.search(r'([가-힣]+(?:동|로|가))', loc_str)
    if match:
        dong_name = match.group(1)
        return dong_to_gu.get(dong_name, "기타")
        
    return "기타"

# 4. 정비사업 데이터 처리
df_plan["일자"] = df_plan["프로젝트코드"].str[8:16]
df_plan["구명"] = df_plan["위치명"].apply(get_gu_name)

# 2022년 이후 & '재개발' 필터링
redev_df = df_plan[(df_plan["일자"] >= "20220101") & (df_plan["소분류"].str.contains("재개발", na=False))].copy()
redev_counts = redev_df["구명"].value_counts().reset_index()
redev_counts.columns = ["자치구명", "재개발건수"]

# 5. 실거래가 데이터 처리 (2024년 이후 상위 25%)
price_recent = price_temp[price_temp['계약일'] >= 20240101].copy()
price_recent['단위면적당금액'] = price_recent['물건금액(만원)'] / price_recent['건물면적(㎡)']
price_stats = price_recent.groupby('자치구명')['단위면적당금액'].quantile(0.75).reset_index()
price_stats.columns = ['자치구명', '상위25%가격']

# 6. 데이터 병합 (how='left'를 사용하여 재개발 건수가 있는 구는 누락되지 않게 함)
final_df = pd.merge(redev_counts, price_stats, on="자치구명", how="left")
final_df = final_df[final_df['자치구명'] != "기타"].dropna(subset=['상위25%가격'])

# 7. 지도 시각화
seoul_geo = {
    '강남구': [37.495, 127.062], '강동구': [37.549, 127.146], '강북구': [37.642, 127.011],
    '강서구': [37.561, 126.825], '관악구': [37.465, 126.943], '광진구': [37.546, 127.085],
    '구로구': [37.494, 126.856], '금천구': [37.460, 126.900], '노원구': [37.652, 127.075],
    '도봉구': [37.669, 127.032], '동대문구': [37.581, 127.054], '동작구': [37.502, 126.951],
    '마포구': [37.559, 126.908], '서대문구': [37.577, 126.939], '서초구': [37.473, 127.031],
    '성동구': [37.551, 127.041], '성북구': [37.605, 127.017], '송파구': [37.505, 127.115],
    '양천구': [37.524, 126.855], '영등포구': [37.522, 126.910], '용산구': [37.530, 126.980],
    '은평구': [37.619, 126.927], '종로구': [37.594, 126.977], '중구': [37.560, 126.995], '중랑구': [37.597, 127.092]
}

m = folium.Map(location=[37.563, 126.986], zoom_start=11, tiles='cartodbpositron')
norm = colors.Normalize(vmin=final_df['상위25%가격'].min(), vmax=final_df['상위25%가격'].max())
cmap = cm.get_cmap('YlOrRd')

for _, row in final_df.iterrows():
    gu = row['자치구명']
    if gu in seoul_geo:
        color_hex = colors.to_hex(cmap(norm(row['상위25%가격'])))
        radius_val = row['재개발건수'] * 3 + 3 
        
        folium.CircleMarker(
            location=seoul_geo[gu],
            radius=radius_val,
            color=color_hex,
            fill=True,
            fill_color=color_hex,
            fill_opacity=0.7,
            popup=f"<b>{gu}</b><br>재개발(22-25): {int(row['재개발건수'])}건<br>상위 25% 평당가: {(row['상위25%가격']*3.3):.0f}만원",
        ).add_to(m)
        
        folium.Marker(
            location=seoul_geo[gu],
            icon=folium.DivIcon(html=f'<div style="font-size: 9pt; font-weight: bold; color: black; text-align: center; white-space: nowrap; width: 100px; margin-left: -50px;">{gu}</div>')
        ).add_to(m)

m.save('seoul_redevelopment_fixed_final.html')
m

In [10]:
import pandas as pd
import folium
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm

# 1. 법정동-자치구 매핑 사전 생성 (무시되는 동 방지용)
# 실거래가 데이터에서 동-구 관계를 추출하여 정비사업의 '동' 데이터를 '구'로 바꿈.
price_temp = pd.read_csv("서울시 부동산 실거래가 정보(2022~2025).csv", encoding="cp949", low_memory=False)
dong_to_gu = price_temp[['법정동명', '자치구명']].drop_duplicates().set_index('법정동명')['자치구명'].to_dict()

# 2. 정비사업 데이터 처리
df_plan = pd.read_csv("서울시 도시계획 정비사업 현황.csv", encoding="cp949")

# 소분류 결측값 찾았지만 재개발 건인지 알 수 없어 제외
df_plan[df_plan['소분류'].isnull()]
# 위치명 결측값 찾았지만 소분류가 재개발이 아니어서 제외
df_plan[df_plan['소분류'].isnull()]


df_plan["일자"] = df_plan["프로젝트코드"].str[8:16]
df_plan["구명"] = ""

for i in range(len(df_plan)):
    location_val = df_plan.loc[i, "위치명"]
    if pd.notnull(location_val) and str(location_val).strip() != "":
        loc_str = str(location_val).replace("서울특별시", "").replace("서울시", "").strip()
        
        # [Step 1]
        if "영등포동" in loc_str: target_gu = "영등포구"
        elif "노량진동" in loc_str: target_gu = "동작구"
        elif "신림동" in loc_str: target_gu = "관악구"
        elif "중곡동" in loc_str: target_gu = "광진구"
        elif any(x in loc_str for x in ["소공동", "산림동", "입정동", "태평로"]): target_gu = "중구"
        elif "전농동" in loc_str: target_gu = "동대문구"
        elif "금호동" in loc_str: target_gu = "성동구"
        elif "길음동" in loc_str: target_gu = "성북구"
        elif "한강로" in loc_str: target_gu = "용산구"
        elif "상계동" in loc_str: target_gu = "노원구"
        elif "명륜" in loc_str: target_gu = "종로구"
        elif "수색" in loc_str: target_gu = "은평구"
        else:
            # [Step 2] 동/로 이름 추출 시도
            find_idx = loc_str.find(" ")
            if find_idx == -1: find_idx = loc_str.find("동")
            if find_idx == -1: find_idx = loc_str.find("로")
            
            extracted_name = loc_str[:find_idx+1].strip() if find_idx != -1 else loc_str
            
            # [Step 3] 추출된 이름 자치구명인지 확인, 아니면 사전에서 검색
            if "구" in extracted_name and len(extracted_name) <= 4:
                target_gu = extracted_name
            else:
                # 사전(dong_to_gu)에서 동 이름을 찾아 구로 변환
                target_gu = dong_to_gu.get(extracted_name, "기타")
        
        df_plan.at[i, "구명"] = target_gu
    else:
        df_plan.at[i, "구명"] = "NULL"

# 2022년 이후 & '재개발' 필터링
redev_df = df_plan[(df_plan["일자"] >= "20220101") & (df_plan["소분류"].str.contains("재개발", na=False))].copy()
redev_counts = redev_df["구명"].value_counts().reset_index()
redev_counts.columns = ["자치구명", "재개발건수"]

# 3. 실거래가 데이터 처리 (2024년~2025년 데이터 중에서만 상위 25% 가격 추출)
# '계약일' 컬럼이 문자열 형태(YYYYMMDD)라고 가정할 때 필터링

# 실거래가 데이터에서 2024년 이후 데이터만 필터링
price_recent = price_temp[price_temp['계약일'] >= 20240101].copy()

# 단위면적당 금액 계산
price_recent['단위면적당금액'] = price_recent['물건금액(만원)'] / price_recent['건물면적(㎡)']

# 자치구별 상위 25% 가격 산출
price_stats = price_recent.groupby('자치구명')['단위면적당금액'].quantile(0.75).reset_index()
price_stats.columns = ['자치구명', '상위25%가격']

# 4. 데이터 병합
# (재개발 건수는 2022~2025 유지, 가격은 2024~2025 데이터로 매칭)
final_df = pd.merge(redev_counts, price_stats, on="자치구명")

# 5. 지도 시각화
seoul_geo = {
    '강남구': [37.495, 127.062], '강동구': [37.549, 127.146], '강북구': [37.642, 127.011],
    '강서구': [37.561, 126.825], '관악구': [37.465, 126.943], '광진구': [37.546, 127.085],
    '구로구': [37.494, 126.856], '금천구': [37.460, 126.900], '노원구': [37.652, 127.075],
    '도봉구': [37.669, 127.032], '동대문구': [37.581, 127.054], '동작구': [37.502, 126.951],
    '마포구': [37.559, 126.908], '서대문구': [37.577, 126.939], '서초구': [37.473, 127.031],
    '성동구': [37.551, 127.041], '성북구': [37.605, 127.017], '송파구': [37.505, 127.115],
    '양천구': [37.524, 126.855], '영등포구': [37.522, 126.910], '용산구': [37.530, 126.980],
    '은평구': [37.619, 126.927], '종로구': [37.594, 126.977], '중구': [37.560, 126.995], '중랑구': [37.597, 127.092]
}

m = folium.Map(location=[37.563, 126.986], zoom_start=11, tiles='cartodbpositron')
norm = colors.Normalize(vmin=final_df['상위25%가격'].min(), vmax=final_df['상위25%가격'].max())
cmap = cm.get_cmap('YlOrRd')

for _, row in final_df.iterrows():
    gu = row['자치구명']
    if gu in seoul_geo:
        color_hex = colors.to_hex(cmap(norm(row['상위25%가격'])))
        radius_val = row['재개발건수'] * 3 + 3 # 가독성을 위해 크기 보정
        
        folium.CircleMarker(
            location=seoul_geo[gu],
            radius=radius_val,
            color=color_hex,
            fill=True,
            fill_color=color_hex,
            fill_opacity=0.7,
            popup=f"<b>{gu}</b><br>재개발(22-25): {row['재개발건수']}건<br>상위 25% 평당가: {(row['상위25%가격']*3.3):.0f}만원",
        ).add_to(m)
        
        folium.Marker(
            location=seoul_geo[gu],
            icon=folium.DivIcon(html=f'<div style="font-size: 9pt; font-weight: bold; color: black; text-align: center; white-space: nowrap; width: 100px; margin-left: -50px;">{gu}</div>')
        ).add_to(m)

m.save('seoul_redevelopment_2022_2025_fixed.html')
m

### 3. 2의 주장에 대한 근거(2011~2020 과거 데이터)

In [7]:
import pandas as pd
import folium
import numpy as np
import matplotlib.colors as colors
import matplotlib.cm as cm

# 1. 분할된 실거래가 데이터 통합
price_files = ['서울시 부동산 실거래가 정보 (9).csv', '서울시 부동산 실거래가 정보 (10).csv'
]

df_price_list = [] 
for f in price_files:
    temp_df = pd.read_csv(f, encoding='cp949', low_memory=False)
    df_price_list.append(temp_df)
    
df_price_total = pd.concat(df_price_list, ignore_index=True)
df_price_total['계약일_dt'] = pd.to_datetime(df_price_total['계약일'], format='%Y%m%d', errors='coerce')

dong_to_gu = df_price_total[['법정동명', '자치구명']].drop_duplicates().set_index('법정동명')['자치구명'].to_dict()

df_past_price = df_price_total[(df_price_total['계약일_dt'] >= '2011-01-01') & (df_price_total['계약일_dt'] < '2021-12-31')].copy()
df_past_price['단위면적당금액'] = df_past_price['물건금액(만원)'] / df_past_price['건물면적(㎡)']

price_stats = df_past_price.groupby('자치구명')['단위면적당금액'].quantile(0.75).reset_index()
price_stats.columns = ['자치구명', '과거상위25%가격']

# 2. 도시계획 정비사업 데이터 처리
df_plan = pd.read_csv("서울시 도시계획 정비사업 현황.csv", encoding="cp949")
df_plan["일자"] = df_plan["프로젝트코드"].str[8:16]
df_plan["구명"] = ""

for i in range(len(df_plan)):
    location_val = df_plan.loc[i, "위치명"]
    target_gu = "기타" 
    
    if pd.notnull(location_val) and str(location_val).strip() != "":
        loc_str = str(location_val).replace("서울특별시", "").replace("서울시", "").strip()
        
        if "영등포동" in loc_str: target_gu = "영등포구"
        elif "노량진동" in loc_str: target_gu = "동작구"
        elif "신림동" in loc_str: target_gu = "관악구"
        elif "중곡동" in loc_str: target_gu = "광진구"
        elif any(x in loc_str for x in ["소공동", "산림동", "입정동", "태평로"]): target_gu = "중구"
        elif "전농동" in loc_str: target_gu = "동대문구"
        elif "금호동" in loc_str: target_gu = "성동구"
        elif "길음동" in loc_str: target_gu = "성북구"
        elif "한강로" in loc_str: target_gu = "용산구"
        elif "상계동" in loc_str: target_gu = "노원구"
        elif "마포구" in loc_str: target_gu = "마포구"
        elif "명륜3가" in loc_str: target_gu = "종로구"
        elif "수색" in loc_str: target_gu = "은평구"
        else:
            find_idx = loc_str.find(" ")
            if find_idx == -1: find_idx = loc_str.find("동")
            if find_idx == -1: find_idx = loc_str.find("로")
            extracted_name = loc_str[:find_idx+1].strip() if find_idx != -1 else loc_str
            if "구" in extracted_name and len(extracted_name) <= 4:
                target_gu = extracted_name
            else:
                target_gu = dong_to_gu.get(extracted_name, "기타")
        df_plan.at[i, "구명"] = target_gu
    else:
        df_plan.at[i, "구명"] = "NULL"

redev_df = df_plan[(df_plan["일자"] >= "20110101") & (df_plan["일자"] <= "20201231") & (df_plan["소분류"].str.contains("재개발", na=False))].copy()
redev_counts = redev_df["구명"].value_counts().reset_index()
redev_counts.columns = ["자치구명", "재개발건수"]

# 3. 최종 데이터 병합
final_df = pd.merge(redev_counts, price_stats, on="자치구명")

# 4. 지도 시각화
seoul_geo = {
    '강남구': [37.495, 127.062], '강동구': [37.549, 127.146], '강북구': [37.642, 127.011],
    '강서구': [37.561, 126.825], '관악구': [37.465, 126.943], '광진구': [37.546, 127.085],
    '구로구': [37.494, 126.856], '금천구': [37.460, 126.900], '노원구': [37.652, 127.075],
    '도봉구': [37.669, 127.032], '동대문구': [37.581, 127.054], '동작구': [37.502, 126.951],
    '마포구': [37.559, 126.908], '서대문구': [37.577, 126.939], '서초구': [37.473, 127.031],
    '성동구': [37.551, 127.041], '성북구': [37.605, 127.017], '송파구': [37.505, 127.115],
    '양천구': [37.524, 126.855], '영등포구': [37.522, 126.910], '용산구': [37.530, 126.980],
    '은평구': [37.619, 126.927], '종로구': [37.594, 126.977], '중구': [37.560, 126.995], '중랑구': [37.597, 127.092]
}

m = folium.Map(location=[37.563, 126.986], zoom_start=11, tiles='cartodbpositron')
norm = colors.Normalize(vmin=final_df['과거상위25%가격'].min(), vmax=final_df['과거상위25%가격'].max())
colormap = cm.get_cmap('YlOrRd')

for _, row in final_df.iterrows():
    gu = row['자치구명']
    if gu in seoul_geo:
        color_hex = colors.to_hex(colormap(norm(row['과거상위25%가격'])))
        radius_val = np.sqrt(row['재개발건수']) * 3 + 5 
        
        folium.CircleMarker(
            location=seoul_geo[gu],
            radius=radius_val,
            color=color_hex,
            fill=True,
            fill_color=color_hex,
            fill_opacity=0.7,
            popup=f"<b>{gu}</b><br>과거 재개발: {row['재개발건수']}건<br>과거 상위 25% 가격: {row['과거상위25%가격']:.1f}만원/㎡",
        ).add_to(m)
        
        #텍스트 강조
        if gu in ['서대문구', '동대문구','성동구','송파구']:
            # 강조 스타일
            label_style = 'font-size: 13pt; font-weight: 900; color: #D32F2F; text-shadow: 1px 1px 2px white;'
        else:
            # 기본 스타일
            label_style = 'font-size: 5pt; font-weight: bold; color: black;'
            
        folium.Marker(
            location=seoul_geo[gu],
            icon=folium.DivIcon(html=f'<div style="{label_style} text-align: center; white-space: nowrap; width: 100px; margin-left: -50px;">{gu}</div>')
        ).add_to(m)

m.save('seoul_past_combined_map.html')
m